<a href="https://colab.research.google.com/github/aaron-abrams-uva/DS1002-S24/blob/main/Pandas/Data_manipulation_instructor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data wrangling with pandas
DS 1002: Programming for Data Science



### PREREQUISITES
- variables
- data types
- operators
- list comprehensions (not essential)
- numpy arrays (not essential)
- earlier pandas notebooks


### SOURCES
- ten minutes to pandas  
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html


- lambda (anonymous) functions  
https://realpython.com/python-lambda/#anonymous-functions


- pivot_table()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html


- concat()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html


- merge()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html


- get_dummies()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html



### OBJECTIVES
- Introduce more advanced pandas dataframe operations for data munging



### CONCEPTS

- DataFrame
- `apply()`
- aggregation using split-apply-combine
- `pivot_table()`
- `groupby()`
- `concat()`
- merging/joining dataframes with `merge()`, `concat()`
- reshaping data
- dummy coding categorical data


---

## Additional Pandas DataFrame Functionality

In earlier pandas notebooks we covered *data frames* (creating, modifying, subsetting, etc) and *cleaning*.  
These notes will demonstrate further methods for data munging and analysis.

In [2]:
import numpy as np
import pandas as pd

#### Load Iris

In [3]:
import seaborn as sns
iris = sns.load_dataset('iris')

In [4]:
iris.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [5]:
# return the size of the object in bytes
import sys
sys.getsizeof(iris)

14744

## Apply Lambda Functions with `.apply()`

Apply a transformation to each record. Uses a `lambda` function.

In [6]:
(lambda x: x**2) (3)

# high_ord_func = lambda x, func: x + func(x)
# high_ord_func(23, lambda x: x * x)


9

In [7]:
iris['sepal_len_sq'] = iris.sepal_length.apply(lambda x: x**2)
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq
0,5.1,3.5,1.4,0.2,setosa,26.01
1,4.9,3.0,1.4,0.2,setosa,24.01
2,4.7,3.2,1.3,0.2,setosa,22.09
3,4.6,3.1,1.5,0.2,setosa,21.16
4,5.0,3.6,1.4,0.2,setosa,25.00


Transformation involving multiple columns. Uses `axis=1` to access columns.  
Compute average of `sepal_length`, `sepal_width`:

In [8]:
iris['sepal_len_wid_avg'] = iris[['sepal_length','sepal_width']].apply(lambda x: (x.sepal_length+x.sepal_width)/2, axis=1)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq,sepal_len_wid_avg
0,5.1,3.5,1.4,0.2,setosa,26.01,4.30
1,4.9,3.0,1.4,0.2,setosa,24.01,3.95
2,4.7,3.2,1.3,0.2,setosa,22.09,3.95
3,4.6,3.1,1.5,0.2,setosa,21.16,3.85
4,5.0,3.6,1.4,0.2,setosa,25.00,4.30


### TRY FOR YOURSELF (UNGRADED EXERCISES)

1) Use `apply()` to append a new column that is the minimum of (petal_length, petal_width)

Print the head, tail of the new dataframe to check things look correct.

In [9]:
iris['min_petal']=iris[['petal_length','petal_width']].apply(lambda x: min(x.petal_length, x.petal_width), axis = 1)
print(iris.head())
print(iris.tail())

   sepal_length  sepal_width  petal_length  petal_width species  sepal_len_sq  \
0           5.1          3.5           1.4          0.2  setosa         26.01   
1           4.9          3.0           1.4          0.2  setosa         24.01   
2           4.7          3.2           1.3          0.2  setosa         22.09   
3           4.6          3.1           1.5          0.2  setosa         21.16   
4           5.0          3.6           1.4          0.2  setosa         25.00   

   sepal_len_wid_avg  min_petal  
0               4.30        0.2  
1               3.95        0.2  
2               3.95        0.2  
3               3.85        0.2  
4               4.30        0.2  
     sepal_length  sepal_width  petal_length  petal_width    species  \
145           6.7          3.0           5.2          2.3  virginica   
146           6.3          2.5           5.0          1.9  virginica   
147           6.5          3.0           5.2          2.0  virginica   
148           6.2    

## Aggregation

Involves one or more of:

- splitting the data into groups
- applying a function to each group
- combining results

### `.groupby()`

Compute mean of each column, grouped (separately) by species

In [10]:
iris.groupby("species").mean()

,sepal_length,sepal_width,petal_length,petal_width,sepal_len_sq,sepal_len_wid_avg,min_petal
species,,,,,,,
setosa,5.006,3.428,1.462,0.246,25.1818,4.217,0.246
versicolor,5.936,2.770,4.260,1.326,35.4972,4.353,1.326
virginica,6.588,2.974,5.552,2.026,43.7980,4.781,2.026


### `pd.pivot_table()`

Apply a function `aggfunc` to selected values grouped by columns

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

In [11]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

print(df)
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                       columns=['C'], aggfunc="sum")
table

     A    B      C  D  E
0  foo  one  small  1  2
1  foo  one  large  2  4
2  foo  one  large  2  5
3  foo  two  small  3  5
4  foo  two  small  3  6
5  bar  one  large  4  6
6  bar  one  small  5  8
7  bar  two  small  6  9
8  bar  two  large  7  9


C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

Compute mean sepal length for each species:

In [12]:
pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = np.mean)

species,setosa,versicolor,virginica
sepal_length,5.006,5.936,6.588


### TRY FOR YOURSELF (UNGRADED EXERCISES)

2) Use a pivot table to compute the following statistics on sepal_width and petal_width grouped by species:

- median  
- mean

These can be computed together in a single call to `pivot_table()`.

In [13]:
pd.pivot_table(iris, values=["sepal_width","petal_width"], columns=["species"], aggfunc = {np.mean, np.median})

species             setosa  versicolor  virginica
petal_width mean     0.246       1.326      2.026
            median   0.200       1.300      2.000
sepal_width mean     3.428       2.770      2.974
            median   3.400       2.800      3.000

## Stacking and Unstacking

Similar to pivoting, but requires -- and takes advantage of -- indexes.

In [14]:
iris_w_idx = iris.copy()

# Give the original index a name
iris_w_idx.index.name = 'obs_id'

# Create a multi-index, using `species` as part of the key.
iris_w_idx = iris_w_idx.reset_index().set_index(['species','obs_id'])

In [15]:
iris_w_idx

sepal_length  sepal_width  petal_length  petal_width  \
species   obs_id                                                         
setosa    0                5.1          3.5           1.4          0.2   
          1                4.9          3.0           1.4          0.2   
          2                4.7          3.2           1.3          0.2   
          3                4.6          3.1           1.5          0.2   
          4                5.0          3.6           1.4          0.2   
...                        ...          ...           ...          ...   
virginica 145              6.7          3.0           5.2          2.3   
          146              6.3          2.5           5.0          1.9   
          147              6.5          3.0           5.2          2.0   
          148              6.2          3.4           5.4          2.3   
          149              5.9          3.0           5.1          1.8   

                  sepal_len_sq  sepal_len_wid_avg  min_petal  
species   obs_id                                              
setosa    0              26.01               4.30        0.2  
          1              24.01               3.95        0.2  
          2              22.09               3.95        0.2  
          3              21.16               3.85        0.2  
          4              25.00               4.30        0.2  
...                        ...                ...        ...  
virginica 145            44.89               4.85        2.3  
          146            39.69               4.40        1.9  
          147            42.25               4.75        2.0  
          148            38.44               4.80        2.3  
          149            34.81               4.45        1.8  

[150 rows x 7 columns]

## `.unstack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html)

In [16]:
iris_wide = iris_w_idx.sepal_length.unstack(fill_value=0).T

In [17]:
iris_wide

species,setosa,versicolor,virginica
obs_id,,,
0,5.1,0.0,0.0
1,4.9,0.0,0.0
2,4.7,0.0,0.0
3,4.6,0.0,0.0
4,5.0,0.0,0.0
...,...,...,...
145,0.0,0.0,6.7
146,0.0,0.0,6.3
147,0.0,0.0,6.5


## `.stack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html)

In [18]:
iris_wide.T.stack().to_frame('sepal_length')

sepal_length
species   obs_id              
setosa    0                5.1
          1                4.9
          2                4.7
          3                4.6
          4                5.0
...                        ...
virginica 145              6.7
          146              6.3
          147              6.5
          148              6.2
          149              5.9

[450 rows x 1 columns]

## Combining DataFrames


### `pd.concat()`  

Concatenate pandas objects along an axis

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

Create two dfs and vertically stack them

In [19]:
df1 = pd.DataFrame(np.random.randn(3, 4))
df2 = pd.DataFrame(np.random.randn(3, 4))

print(df1)
print('-'*45)
print(df2)

df3 = pd.concat([df1, df2], axis=0)

print('-'*45)
print(df3)

          0         1         2         3
0 -0.580766  0.294242 -1.310028 -1.775874
1  0.056212 -0.029240 -0.658659 -0.268010
2 -1.300260  1.792054 -1.497236  0.253376
---------------------------------------------
          0         1         2         3
0  0.564332 -0.650007 -0.292896  0.917390
1  0.973567  1.725304 -0.025683  0.226235
2 -0.439225 -0.128073  0.664898 -0.721310
---------------------------------------------
          0         1         2         3
0 -0.580766  0.294242 -1.310028 -1.775874
1  0.056212 -0.029240 -0.658659 -0.268010
2 -1.300260  1.792054 -1.497236  0.253376
0  0.564332 -0.650007 -0.292896  0.917390
1  0.973567  1.725304 -0.025683  0.226235
2 -0.439225 -0.128073  0.664898 -0.721310


**Concat columns**  
This assumes that the indexes represent IDs of specific things or events

In [20]:
df4 = pd.concat([df1,df2], axis = 1, keys = ['foo', 'bar'])

df4

foo                                     bar                      \
          0         1         2         3         0         1         2   
0 -0.580766  0.294242 -1.310028 -1.775874  0.564332 -0.650007 -0.292896   
1  0.056212 -0.029240 -0.658659 -0.268010  0.973567  1.725304 -0.025683   
2 -1.300260  1.792054 -1.497236  0.253376 -0.439225 -0.128073  0.664898   

             
          3  
0  0.917390  
1  0.226235  
2 -0.721310

In [21]:
df4.foo

,0,1,2,3
0,-0.580766,0.294242,-1.310028,-1.775874
1,0.056212,-0.029240,-0.658659,-0.268010
2,-1.300260,1.792054,-1.497236,0.253376


In [22]:
df4.bar

,0,1,2,3
0,0.564332,-0.650007,-0.292896,0.917390
1,0.973567,1.725304,-0.025683,0.226235
2,-0.439225,-0.128073,0.664898,-0.721310


### `merge()`

SQL-style joining of tables (DataFrames)

Important parameters include:

- `how` : type of merge {'left', 'right', 'outer', 'inner', 'cross'}, default ‘inner’
- `on`  : names to join on
        
[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Create two tables, `left` and `right`. Then right join them on `key`.  
Right join means include all records from table on right.  
The `key` is used for matching up the records.

In [23]:
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  lval
0  jamie    15
1   bill    22

---right
     key  rval
0  jamie     4
1   bill     5
2  asher     8

---joined
     key  lval  rval
0  jamie  15.0     4
1   bill  22.0     5
2  asher   NaN     8


Notice the NaN inserted into the record with key=asher, since the left table didn't contain the key.

**Matching column names**  
In this next example, the value columns have the same name: *val*.  Notice what happens to the column names.

In [24]:
left = pd.DataFrame({"key": ["jamie", "bill"], "val": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "val": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  val
0  jamie   15
1   bill   22

---right
     key  val
0  jamie    4
1   bill    5
2  asher    8

---joined
     key  val_x  val_y
0  jamie   15.0      4
1   bill   22.0      5
2  asher    NaN      8


## `.join()`

An SQL-like joiner, but this one takes advantage of indexes.

Give our dataframes indexes and distinctive columns names.

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)

In [25]:
left2 = left.set_index('key').rename(columns={'val':'val_1'})
right2 = right.set_index('key').rename(columns={'val':'val_2'})

In [26]:
left2

,val_1
key,
jamie,15
bill,22


In [27]:
right2

,val_2
key,
jamie,4
bill,5
asher,8


In [28]:
right2.join(left2) # defaults to first df (right2 in this case)

,val_2,val_1
key,,
jamie,4,15.0
bill,5,22.0
asher,8,NaN


In [29]:
right2.join(left2, how = 'inner')

,val_2,val_1
key,,
jamie,4,15
bill,5,22


## Summary

* Use **join** if you have shared indexes
* Use **merge** if you do not have shared indexes
* Use **concat** to combine based on shared indexes or columns

### TRY FOR YOURSELF (UNGRADED EXERCISES)

3) Redo the join exercise above, using an inner join instead of a right join.  
Make sure the results make sense.

In [30]:
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="inner")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  lval
0  jamie    15
1   bill    22

---right
     key  rval
0  jamie     4
1   bill     5
2  asher     8

---joined
     key  lval  rval
0  jamie    15     4
1   bill    22     5


## `.reshape()`

Changes the object's shape

We illustrate creating pandas Series, extracting array of length 6, and reshaping to 3x2 array.

In [31]:
# create a series
ser = pd.Series([1, 1, 2, 3, 5, 8])

# extract values
vals = ser.values

print('orig data:', vals)
print('orig type:', type(vals))
print('orig shape:', vals.shape)

# reshaping series
reshaped_vals = vals.reshape((3, 2))

print('\n reshaped vals:')
print(reshaped_vals)
print('\n new type:', type(reshaped_vals))
print('new shape:', reshaped_vals.shape)

orig data: [1 1 2 3 5 8]
orig type: <class 'numpy.ndarray'>
orig shape: (6,)

 reshaped vals:
[[1 1]
 [2 3]
 [5 8]]

 new type: <class 'numpy.ndarray'>
new shape: (3, 2)


Including -1 as one of the dimensions tells numpy: infer this dimension from the data and the other dimensions.

Example: enforce 3 columns:

In [32]:
vals.reshape(-1,3)

array([[1, 1, 2],
       [3, 5, 8]])

Enforce 3 rows:

In [33]:
vals.reshape(3,-1)

array([[1, 1],
       [2, 3],
       [5, 8]])

**IMPORTANT NOTE**  

Notice the shape of original array: `(6,)`  
This is a vector with one dimension, and is different from two-dimensional `(6,1)` array

### TRY FOR YOURSELF (UNGRADED EXERCISES)

4) Recreate the series from above with data [1, 1, 2, 3, 5, 8]  
Extract the data from the series and reshape to 2x3.  
Print both the reshaped data, and the dimensions.

In [34]:
ser = pd.Series([1, 1, 2, 3, 5, 8])
vals= ser.values
resh = vals.reshape(2,3)
print(resh)
print('new shape:', resh.shape)

[[1 1 2]
 [3 5 8]]
new shape: (2, 3)


## Categoricals

Categorical data takes discrete values where computation on the values does not make sense.  
Zip code is a typical example.

To include categoricals in models, they must be converted to numeric.  

### `get_dummies()`
Dummy code categorical data

The parameter `prefix` appends the prefix to column names (a good idea for later use)
<!-- - `drop_first`: remove first level, as only `k-1` variables needed to represent `k` levels
-->

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

In [35]:
cats = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese']})

print('--categorical data')
print(cats)

cats = pd.get_dummies(cats.breed, prefix='breed')

print('\n')
print('--dummified categorical data')
print(cats)

--categorical data
       breed
0    persian
1    persian
2    siamese
3  himalayan
4    burmese


--dummified categorical data
   breed_burmese  breed_himalayan  breed_persian  breed_siamese
0              0                0              1              0
1              0                0              1              0
2              0                0              0              1
3              0                1              0              0
4              1                0              0              0


Often the first level (variable) is dropped, because it is redundant information and can be inferred from the remaining columns.  To do this include `drop_first = True` as an argument to the `get_dummies()` call.

For certain types of statistical analysis (regressions etc) it is helpful to do this.
See for example https://datascience.stackexchange.com/questions/27957/why-do-we-need-to-discard-one-dummy-variable/27993#27993

### TRY FOR YOURSELF (UNGRADED EXERCISES)

5) The dataframe below contains two categoricals. Dummify each of them, giving them a prefix and dropping the first level from each.

Print the new dataframe to insure correctness.

Hint: You might want to dummify each column into separate new dataframes, and then merge them together by using:

`pd.concat([df1, df2], axis=1)`

In [46]:
cats2 = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese'],
                      'color':['calico','white','seal point','cream','sable']})

In [48]:
cats2 = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese'],
                      'color':['calico','white','seal point','cream','sable']})

breed = pd.get_dummies(cats2.breed, drop_first=True, prefix='breed_')
color = pd.get_dummies(cats2.color, drop_first=True, prefix='color_')

cats3 = pd.concat([breed, color], axis=1)
cats3

       breed       color
0    persian      calico
1    persian       white
2    siamese  seal point
3  himalayan       cream
4    burmese       sable


,breed__himalayan,breed__persian,breed__siamese,color__cream,color__sable,color__seal point,color__white
0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,1
2,0,0,1,0,0,1,0
3,1,0,0,1,0,0,0
4,0,0,0,0,1,0,0


Note that in the above solution we have used the `drop_first = True` argument, so the first variable (in each of the two separate data frames) is gone.

---